In [5]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
import os
from pydantic import BaseModel

In [3]:
load_dotenv(override=True)

True

In [6]:
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
    

OpenAI API Key exists and begins sk-proj-
